Goal of this notebook:

Investigate the impact of using discretized datasets (log2, log10, and mdlp) on the classification accuracy with decision tree, random forest and XGBoost.

In [1]:
import numpy as np
import pandas as pd
from IPython.core.display import display

pd.set_option('display.max_columns', None)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRFClassifier
import warnings
import classifier_utils
import configs
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
non_features_columns = ["chunk_id", "line_start", "line_end", "line_separator", "kind_conflict", "url", "project"]
non_features_columns.extend(["project_user", "project_name", "path", "file_name", "sha", "leftsha", "rightsha", "basesha"])

In [3]:
selected_dataset = pd.read_csv("../../data/SELECTED_LABELLED_DATASET.csv")
projects = list(selected_dataset['project'].unique())

# Decision Tree

In [4]:
dt = DecisionTreeClassifier(random_state=99)

In [5]:
result_dt_default = classifier_utils.ProjectsResults(dt, projects, non_features_columns)

In [6]:
report_dt_default = result_dt_default.get_report_df(include_overall=True)
# report_dt_default

In [7]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log10'
result_dt_log10 = classifier_utils.ProjectsResults(dt, projects, non_features_columns,discretized_path)
report_dt_log10 = result_dt_log10.get_report_df(include_overall=True)
# report_dt_log10

In [8]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log2'
result_dt_log2 = classifier_utils.ProjectsResults(dt, projects, non_features_columns,discretized_path)
report_dt_log2 = result_dt_log2.get_report_df(include_overall=True)
# report_dt_log2

In [9]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_mdlp'
result_dt_mdlp = classifier_utils.ProjectsResults(dt, projects, non_features_columns,discretized_path)
report_dt_mdlp = result_dt_mdlp.get_report_df(include_overall=True)

In [10]:
df_inner_dt = pd.merge(report_dt_default, report_dt_log10, on='project', how='inner', suffixes=('_default', '_log10'))
df_inner_dt_log2 = report_dt_log2.add_suffix("_log2").rename(columns={"project_log2": "project"})
df_inner_dt = pd.merge(df_inner_dt, df_inner_dt_log2, on='project', how='inner')
df_inner_dt_mdlp = report_dt_mdlp.add_suffix("_mdlp").rename(columns={"project_mdlp": "project"})
df_inner_dt = pd.merge(df_inner_dt, df_inner_dt_mdlp, on='project', how='inner')

accuracy_dt = df_inner_dt.filter(regex=("project|accuracy.*")).copy()
accuracy_dt['improv._log10'] = accuracy_dt.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log10'], x['accuracy_default']), axis=1)
accuracy_dt['improv._log2'] = accuracy_dt.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log2'], x['accuracy_default']), axis=1)
accuracy_dt['improv._mdlp'] = accuracy_dt.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_mdlp'], x['accuracy_default']), axis=1)
accuracy_dt

,project,accuracy_default,accuracy_log10,accuracy_log2,accuracy_mdlp,improv._log10,improv._log2,improv._mdlp
0,CCI-MIT__XCoLab,0.97200,0.96700,0.9720,0.969000,-0.005144,0.000000,-0.003086
1,apache__directory-server,0.92800,0.92000,0.9230,0.885000,-0.008621,-0.005388,-0.046336
2,jgralab__jgralab,0.83900,0.75600,0.8330,0.795000,-0.098927,-0.007151,-0.052443
3,Unidata__thredds,0.90900,0.89900,0.9030,0.849000,-0.011001,-0.006601,-0.066007
4,apache__accumulo,0.83800,0.83100,0.8300,0.829000,-0.008353,-0.009547,-0.010740
5,CloudStack-extras__CloudStack-archive,0.75000,0.75500,0.7560,0.739000,0.020000,0.024000,-0.014667
6,getrailo__railo,0.65000,0.63100,0.6450,0.645000,-0.029231,-0.007692,-0.007692
7,Ramblurr__Anki-Android,0.68200,0.64200,0.6590,0.682000,-0.058651,-0.033724,0.000000
8,TeamDev-Ltd__OpenFaces,0.96400,0.95800,0.9650,0.965000,-0.006224,0.027778,0.027778
9,apache__lucene-solr,0.55500,0.57700,0.5750,0.533000,0.049438,0.044944,-0.039640


## Including NA values

In [11]:
import importlib
importlib.reload(classifier_utils)
result_dt_default_wna = classifier_utils.ProjectsResults(dt, projects, non_features_columns, replace_na=True)

In [12]:
report_dt_default_wna = result_dt_default_wna.get_report_df(include_overall=True)
# report_dt_default_wna

In [13]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log2'
result_dt_log2_wna = classifier_utils.ProjectsResults(dt, projects, non_features_columns,discretized_path, replace_na=True)
report_dt_log2_wna = result_dt_log2_wna.get_report_df(include_overall=True)
# report_dt_log2_wna

In [14]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log10'
result_dt_log10_wna = classifier_utils.ProjectsResults(dt, projects, non_features_columns,discretized_path, replace_na=True)
report_dt_log10_wna = result_dt_log10_wna.get_report_df(include_overall=True)
# report_dt_log10_wna

In [15]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_mdlp'
result_dt_mdlp_wna = classifier_utils.ProjectsResults(dt, projects, non_features_columns,discretized_path, replace_na=True)
report_dt_mdlp_wna = result_dt_mdlp_wna.get_report_df(include_overall=True)
# report_dt_log10_wna

In [16]:
df_inner_dt = pd.merge(report_dt_default_wna, report_dt_log10_wna, on='project', how='inner', suffixes=('_default', '_log10'))
df_inner_dt_log2 = report_dt_log2_wna.add_suffix("_log2").rename(columns={"project_log2": "project"})
df_inner_dt = pd.merge(df_inner_dt, df_inner_dt_log2, on='project', how='inner')
df_inner_dt_mdlp = report_dt_mdlp_wna.add_suffix("_mdlp").rename(columns={"project_mdlp": "project"})
df_inner_dt = pd.merge(df_inner_dt, df_inner_dt_mdlp, on='project', how='inner')

# accuracy_dt = df_inner_dt.filter(regex=("project|observations_default|accuracy.*")).copy()
accuracy_dt = df_inner_dt.filter(regex=("project|accuracy.*")).copy()
accuracy_dt['improv._log10'] = accuracy_dt.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log10'], x['accuracy_default']), axis=1)
accuracy_dt['improv._log2'] = accuracy_dt.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log2'], x['accuracy_default']), axis=1)
accuracy_dt['improv._mdlp'] = accuracy_dt.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_mdlp'], x['accuracy_default']), axis=1)
accuracy_dt

,project,accuracy_default,accuracy_log10,accuracy_log2,accuracy_mdlp,improv._log10,improv._log2,improv._mdlp
0,CCI-MIT__XCoLab,0.971000,0.971000,0.971000,0.969000,0.000000,0.000000,-0.002060
1,apache__directory-server,0.902000,0.890000,0.877000,0.885000,-0.013304,-0.027716,-0.018847
2,jgralab__jgralab,0.823000,0.751000,0.818000,0.795000,-0.087485,-0.006075,-0.034022
3,CloudStack-extras__CloudStack-archive,0.780000,0.763000,0.772000,0.747000,-0.021795,-0.010256,-0.042308
4,Unidata__thredds,0.904000,0.891000,0.893000,0.852000,-0.014381,-0.012168,-0.057522
5,apache__accumulo,0.854000,0.847000,0.849000,0.848000,-0.008197,-0.005855,-0.007026
6,getrailo__railo,0.721000,0.715000,0.724000,0.709000,-0.008322,0.010753,-0.016644
7,Ramblurr__Anki-Android,0.674000,0.675000,0.666000,0.688000,0.003067,-0.011869,0.042945
8,TeamDev-Ltd__OpenFaces,0.952000,0.958000,0.953000,0.955000,0.125000,0.020833,0.062500
9,apache__lucene-solr,0.538000,0.538000,0.537000,0.543000,0.000000,-0.001859,0.010823


The overall average accuracy did not increase using discretized datasets (log2, log10, and mdlp). The same was observed when considering the dataset with and without NA values.

# Random forest

In [17]:
rf = RandomForestClassifier(random_state=99, n_jobs=5, n_estimators=400, max_features=0.3, min_samples_leaf=1)# Decision Tree

In [18]:
result_rf_default = classifier_utils.ProjectsResults(rf, projects, non_features_columns)

In [19]:
report_rf_default = result_rf_default.get_report_df(include_overall=True)
# report_rf_default

In [20]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log10'
result_rf_log10 = classifier_utils.ProjectsResults(rf, projects, non_features_columns,discretized_path)
report_rf_log10 = result_rf_log10.get_report_df(include_overall=True)
# report_rf_log10

In [21]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log2'
result_rf_log2 = classifier_utils.ProjectsResults(rf, projects, non_features_columns,discretized_path)
report_rf_log2 = result_rf_log2.get_report_df(include_overall=True)
# report_dt_log2

In [22]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_mdlp'
result_rf_mdlp = classifier_utils.ProjectsResults(rf, projects, non_features_columns,discretized_path)
report_rf_mdlp = result_rf_mdlp.get_report_df(include_overall=True)
# report_dt_log2

In [23]:
df_inner_rf = pd.merge(report_rf_default, report_rf_log10, on='project', how='inner', suffixes=('_default', '_log10'))
df_inner_rf_log2 = report_rf_log2.add_suffix("_log2").rename(columns={"project_log2": "project"})
df_inner_rf = pd.merge(df_inner_rf, df_inner_rf_log2, on='project', how='inner')
df_inner_rf_mdlp = report_rf_mdlp.add_suffix("_mdlp").rename(columns={"project_mdlp": "project"})
df_inner_rf = pd.merge(df_inner_rf, df_inner_rf_mdlp, on='project', how='inner')

accuracy_rf = df_inner_rf.filter(regex=("project|accuracy.*")).copy()
accuracy_rf['improv._log10'] = accuracy_rf.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log10'], x['accuracy_default']), axis=1)
accuracy_rf['improv._log2'] = accuracy_rf.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log2'], x['accuracy_default']), axis=1)
accuracy_rf['improv._mdlp'] = accuracy_rf.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_mdlp'], x['accuracy_default']), axis=1)
accuracy_rf

,project,accuracy_default,accuracy_log10,accuracy_log2,accuracy_mdlp,improv._log10,improv._log2,improv._mdlp
0,CCI-MIT__XCoLab,0.97600,0.9740,0.9750,0.97300,-0.002049,-0.001025,-0.003074
1,apache__directory-server,0.93700,0.9230,0.9330,0.91400,-0.014941,-0.004269,-0.024546
2,jgralab__jgralab,0.86600,0.8140,0.8670,0.83300,-0.060046,0.007463,-0.038106
3,CloudStack-extras__CloudStack-archive,0.80600,0.8030,0.8050,0.77100,-0.003722,-0.001241,-0.043424
4,apache__accumulo,0.86300,0.8650,0.8660,0.85500,0.014599,0.021898,-0.009270
5,Unidata__thredds,0.91600,0.9260,0.9210,0.86200,0.119048,0.059524,-0.058952
6,Ramblurr__Anki-Android,0.74200,0.7340,0.7420,0.71500,-0.010782,0.000000,-0.036388
7,getrailo__railo,0.71200,0.6920,0.7110,0.70300,-0.028090,-0.001404,-0.012640
8,apache__lucene-solr,0.64600,0.6420,0.6520,0.58600,-0.006192,0.016949,-0.092879
9,TeamDev-Ltd__OpenFaces,0.96900,0.9680,0.9690,0.97100,-0.001032,0.000000,0.064516


## Including NA values

In [24]:
import importlib
importlib.reload(classifier_utils)
result_rf_default_wna = classifier_utils.ProjectsResults(rf, projects, non_features_columns, replace_na=True)

In [25]:
report_rf_default_wna = result_rf_default_wna.get_report_df(include_overall=True)
# report_dt_default_wna

In [26]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log2'
result_rf_log2_wna = classifier_utils.ProjectsResults(rf, projects, non_features_columns,discretized_path, replace_na=True)
report_rf_log2_wna = result_rf_log2_wna.get_report_df(include_overall=True)
# report_dt_log2_wna

In [27]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log10'
result_rf_log10_wna = classifier_utils.ProjectsResults(rf, projects, non_features_columns,discretized_path, replace_na=True)
report_rf_log10_wna = result_rf_log10_wna.get_report_df(include_overall=True)
# report_dt_log10_wna

In [28]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_mdlp'
result_rf_mdlp_wna = classifier_utils.ProjectsResults(rf, projects, non_features_columns,discretized_path, replace_na=True)
report_rf_mdlp_wna = result_rf_mdlp_wna.get_report_df(include_overall=True)
# report_dt_log10_wna

In [29]:
df_inner_rf = pd.merge(report_rf_default_wna, report_rf_log10_wna, on='project', how='inner', suffixes=('_default', '_log10'))
df_inner_rf_log2 = report_rf_log2_wna.add_suffix("_log2").rename(columns={"project_log2": "project"})
df_inner_rf = pd.merge(df_inner_rf, df_inner_rf_log2, on='project', how='inner')
df_inner_rf_mdlp = report_rf_mdlp_wna.add_suffix("_mdlp").rename(columns={"project_mdlp": "project"})
df_inner_rf = pd.merge(df_inner_rf, df_inner_rf_mdlp, on='project', how='inner')

# accuracy_dt = df_inner_dt.filter(regex=("project|observations_default|accuracy.*")).copy()
accuracy_rf = df_inner_rf.filter(regex=("project|accuracy.*")).copy()
accuracy_rf['improv._log10'] = accuracy_rf.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log10'], x['accuracy_default']), axis=1)
accuracy_rf['improv._log2'] = accuracy_rf.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log2'], x['accuracy_default']), axis=1)
accuracy_rf['improv._mdlp'] = accuracy_rf.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_mdlp'], x['accuracy_default']), axis=1)
accuracy_rf

,project,accuracy_default,accuracy_log10,accuracy_log2,accuracy_mdlp,improv._log10,improv._log2,improv._mdlp
0,CCI-MIT__XCoLab,0.977000,0.975000,0.97600,0.975000,-0.002047,-0.001024,-0.002047
1,apache__directory-server,0.931000,0.924000,0.92700,0.914000,-0.007519,-0.004296,-0.018260
2,jgralab__jgralab,0.868000,0.816000,0.86100,0.833000,-0.059908,-0.008065,-0.040323
3,CloudStack-extras__CloudStack-archive,0.822000,0.824000,0.82000,0.786000,0.011236,-0.002433,-0.043796
4,Unidata__thredds,0.913000,0.923000,0.91800,0.878000,0.114943,0.057471,-0.038335
5,apache__accumulo,0.879000,0.875000,0.88000,0.872000,-0.004551,0.008264,-0.007964
6,getrailo__railo,0.779000,0.766000,0.77700,0.766000,-0.016688,-0.002567,-0.016688
7,Ramblurr__Anki-Android,0.763000,0.748000,0.74800,0.726000,-0.019659,-0.019659,-0.048493
8,TeamDev-Ltd__OpenFaces,0.962000,0.961000,0.95900,0.961000,-0.001040,-0.003119,-0.001040
9,apache__lucene-solr,0.640000,0.623000,0.64800,0.584000,-0.026563,0.022222,-0.087500


The overall average accuracy did not increase using discretized datasets (log2, log10, and mdlp). The same was observed when considering the dataset with and without NA values.

# XGBoost Random Forest

In [30]:
xg = XGBRFClassifier(random_state=99, subsample=0.9, eval_metric='mlogloss', n_estimators=400, colsample_bynode=0.4)

In [31]:
result_xg_default = classifier_utils.ProjectsResults(xg, projects, non_features_columns, drop_na=False)

In [32]:
report_xg_default = result_xg_default.get_report_df(include_overall=True)
# report_rf_default

In [33]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log10'
result_xg_log10 = classifier_utils.ProjectsResults(xg, projects, non_features_columns,discretized_path, drop_na=False)
report_xg_log10 = result_xg_log10.get_report_df(include_overall=True)
# report_rf_log10

In [34]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log2'
result_xg_log2 = classifier_utils.ProjectsResults(xg, projects, non_features_columns,discretized_path, drop_na=False)
report_xg_log2 = result_xg_log2.get_report_df(include_overall=True)
# report_dt_log2

In [35]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_mdlp'
result_xg_mdlp = classifier_utils.ProjectsResults(xg, projects, non_features_columns,discretized_path, drop_na=False)
report_xg_mdlp = result_xg_mdlp.get_report_df(include_overall=True)
# report_dt_log2

In [36]:
df_inner_xg = pd.merge(report_xg_default, report_xg_log10, on='project', how='inner', suffixes=('_default', '_log10'))
df_inner_xg_log2 = report_xg_log2.add_suffix("_log2").rename(columns={"project_log2": "project"})
df_inner_xg = pd.merge(df_inner_xg, df_inner_xg_log2, on='project', how='inner')
df_inner_xg_mdlp = report_xg_mdlp.add_suffix("_mdlp").rename(columns={"project_mdlp": "project"})
df_inner_xg = pd.merge(df_inner_xg, df_inner_xg_mdlp, on='project', how='inner')

accuracy_xg = df_inner_xg.filter(regex=("project|accuracy.*")).copy()
accuracy_xg['improv._log10'] = accuracy_xg.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log10'], x['accuracy_default']), axis=1)
accuracy_xg['improv._log2'] = accuracy_xg.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log2'], x['accuracy_default']), axis=1)
accuracy_xg['improv._mdlp'] = accuracy_xg.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_mdlp'], x['accuracy_default']), axis=1)
accuracy_xg

,project,accuracy_default,accuracy_log10,accuracy_log2,accuracy_mdlp,improv._log10,improv._log2,improv._mdlp
0,CCI-MIT__XCoLab,0.974000,0.972000,0.972000,0.97400,-0.002053,-0.002053,0.000000
1,apache__directory-server,0.916000,0.909000,0.909000,0.89600,-0.007642,-0.007642,-0.021834
2,jgralab__jgralab,0.816000,0.720000,0.797000,0.78300,-0.117647,-0.023284,-0.040441
3,Unidata__thredds,0.904000,0.899000,0.902000,0.84700,-0.005531,-0.002212,-0.063053
4,getrailo__railo,0.766000,0.745000,0.761000,0.74000,-0.027415,-0.006527,-0.033943
5,apache__accumulo,0.856000,0.841000,0.850000,0.85200,-0.017523,-0.007009,-0.004673
6,Ramblurr__Anki-Android,0.717000,0.713000,0.706000,0.71500,-0.005579,-0.015342,-0.002789
7,TeamDev-Ltd__OpenFaces,0.962000,0.958000,0.959000,0.95900,-0.004158,-0.003119,-0.003119
8,CloudStack-extras__CloudStack-archive,0.706000,0.703000,0.696000,0.67500,-0.004249,-0.014164,-0.043909
9,apache__lucene-solr,0.595000,0.581000,0.592000,0.58000,-0.023529,-0.005042,-0.025210


The overall average accuracy did not increase using discretized datasets (log2, log10, and mdlp). The same was observed when considering the dataset with and without NA values.